In [57]:
import tabula
import numpy as np
import pandas as pd
import os 
from pathlib import Path
import PyPDF2
import re
import requests
import json

In [92]:
filenames = [
             os.path.expanduser('/home/parth/Documents/USICT/it_res.pdf'),
             os.path.expanduser('/home/parth/Documents/USICT/cse_res.pdf'),
             os.path.expanduser('/home/parth/Documents/USICT/ece_res.pdf')]

In [93]:
scheme_reg = re.compile(r'scheme\s+of\s+examinations',re.IGNORECASE)
institution_reg = re.compile(r'institution\s*:\s*([\w\n(,)& ]+)\nS\.No',re.IGNORECASE)
sem_reg = re.compile(r'se\s?m[.//\w\n]+:\s+([\w\n]+)',re.IGNORECASE)
programme_reg = re.compile(r'programme\s+name:\s+([\w(,)& \n]+)SchemeID',re.IGNORECASE)
branch_reg = re.compile(r'[\w &]+\(([\w ]+)\)')

In [90]:
def get_info(text) :
    college = institution_reg.search(text)[1].replace('\n','').strip().title()
    semester = int(sem_reg.search(text)[1].replace('\n','').strip())
    course = programme_reg.search(text)[1].replace('\n','').strip().title()
    branch = branch_reg.search(course)[1].strip().title()
    course = course[0:course.find('(')].strip()
    info = {
        'college' : college,
        'semester' : semester,
        'course' : course,
        'branch' : branch,
    }
    return info

In [96]:
# SITE = "https://cgpa-book.herokuapp.com/academia"
SITE = "http://localhost:3000/academia"

In [98]:
#Add college
data ={ 
    'college' : {
        'college' : "University School Of Information, Communication & Technology (Formerly Usit)"
        }
}
r = requests.post(SITE+"/college",json=data)
print(r,r.content)


<Response [200]> b'OK'


In [99]:
def already_exists(info) :
    r = requests.get(SITE+"/semester",json=info)
    content = json.loads(r.content)
#     print(r.status_code,r.content)
    return r.status_code == 200 and content != {}
    

In [100]:
def getSubjects(df) :
    subjects = []
    for index,row in df.iterrows() :
        subject = {}
        subject['subject'] = row['Subject'].strip().title()
        subject['subjectCode'] = row['Code']
        subject['credits'] = row['Credit']
        subjects.append(subject)
    return subjects

In [ ]:
for filename in filenames :
    pdf = PyPDF2.PdfFileReader(filename)
    print(filename,pdf.getNumPages())
    
    for i in range(0,pdf.getNumPages()) :
        text = pdf.getPage(i).extractText()
        if scheme_reg.search(text) :
            info = get_info(text)
            df = tabula.read_pdf(filename,pages=i+1)
            subjects = getSubjects(df[0])        
            if already_exists(info) :
                print("information already exists")
                continue
            info['semester'] = {'semester' : info['semester'], 'subjects' : subjects}
            r = requests.post(SITE+"/semester",json=info)
            print(r,r.content)
#             print(info)

    

/home/parth/Documents/USICT/it_res.pdf 58
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'
<Response [400]> b'{"type":"ValueError","message":"College validation failed: courses.0.branches.0.semesters: Semester already exists"}'

In [ ]:
data = {
    'college' : 'University School Of Information, Communication & Technology (Formerly Usit)',
    'course' : {
        'course' : 'Bachelor Of Technology'
    }
}
r = requests.post(SITE+'/course',json=data)
print(r.status_code,r.content)

In [ ]:
from IPython.display import display,HTML

In [ ]:
dfs = tabula.read_pdf(filename,pages='all')

In [ ]:
dd'Subject' in dfs[0].columns

In [ ]:
display(dfs[0])

In [ ]:
t